In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

archivo_excel = "/content/Datos_Restaurante_Ejemplo_6meses.xlsx"
xls = pd.ExcelFile(archivo_excel)

# Cargar las hojas necesarias
ventas = xls.parse('Ventas')
detalles = xls.parse('DetallesVenta')
platillos = xls.parse('Platillos')


In [ ]:
total_clientes = ventas['id_cliente'].nunique()
print("Total de Clientes:", total_clientes)

Total de Clientes: 60


In [ ]:
ticket_promedio = ventas['total'].mean()
print("Ticket Promedio: $", round(ticket_promedio, 2))

Ticket Promedio: $ 56.28


In [ ]:
visitas_por_cliente = ventas.groupby('id_cliente').size()
promedio_visitas = visitas_por_cliente.mean()
print("Promedio de visitas por cliente:", round(promedio_visitas, 1))

Promedio de visitas por cliente: 8.6


In [ ]:
clientes = xls.parse('Clientes')
# Calcular el gasto total por cliente
gasto_por_cliente = ventas.groupby('id_cliente')['total'].sum().reset_index(name='gasto_total')

# Obtener el top 5
top_gasto = gasto_por_cliente.sort_values(by='gasto_total', ascending=False).head(5)

# Unir con el nombre del cliente
top_gasto = top_gasto.merge(clientes[['id_cliente', 'nombre_cliente']], on='id_cliente')

# Reordenar columnas para mejor visualización
top_gasto = top_gasto[['nombre_cliente', 'id_cliente', 'gasto_total']]

# Mostrar como tabla formateada
top_gasto.style.format({'gasto_total': '${:,.2f}'}).set_caption("Top 5 Clientes por Gasto")

,nombre_cliente,id_cliente,gasto_total
0,David Herrera,9,$873.00
1,Francisco Molina,24,$867.10
2,Rosario Guzmán,40,$837.45
3,Verónica León,16,$751.00
4,Claudia Vargas,18,$750.60


In [ ]:
clientes = xls.parse('Clientes')

# Unir detalles con ventas para conocer el cliente de cada detalle
detalles_con_cliente = detalles.merge(ventas[['id_venta', 'id_cliente']], on='id_venta')

# Unir con los nombres de platillos
detalles_completos = detalles_con_cliente.merge(platillos[['id_platillo', 'nombre_platillo']], on='id_platillo')

# Total de compras por cliente
compras_por_cliente = ventas.groupby('id_cliente').size().reset_index(name='Compras')

# Plato más comprado por cliente
plato_favorito = (
    detalles_completos.groupby(['id_cliente', 'nombre_platillo'])['cantidad']
    .sum()
    .reset_index()
    .sort_values(['id_cliente', 'cantidad'], ascending=[True, False])
)

# Obtener solo el plato favorito por cliente
plato_favorito_unico = plato_favorito.groupby('id_cliente').first().reset_index()

# Combinar con compras
tabla_compras_favorito = compras_por_cliente.merge(plato_favorito_unico, on='id_cliente')

# Agregar el nombre del cliente desde la tabla de clientes
tabla_compras_favorito = tabla_compras_favorito.merge(clientes[['id_cliente', 'nombre_cliente']], on='id_cliente')

# Ordenar por número de compras descendente y tomar solo los primeros 5 clientes
top5 = tabla_compras_favorito.sort_values(by='Compras', ascending=False).head(5)


# Opcional: aplicar estilo para visualización limpia en Jupyter o entorno compatible
top5.style.set_caption("Top 5 Clientes por Número de Compras")


,id_cliente,Compras,nombre_platillo,cantidad,nombre_cliente
13,14,16,Hamburguesa clásica,10,Pablo Romero
15,16,15,Brownie con nuez,12,Verónica León
39,40,15,Tacos al pastor,12,Rosario Guzmán
8,9,14,Burrito de carne,12,David Herrera
23,24,14,Hamburguesa clásica,14,Francisco Molina


In [ ]:
# Calcular el ticket promedio por cliente
ticket_promedio_por_cliente = ventas.groupby('id_cliente')['total'].mean().reset_index(name='ticket_promedio')

# Ordenar y quedarte con el top 5
top_ticket_promedio = ticket_promedio_por_cliente.sort_values(by='ticket_promedio', ascending=False).head(5)

# Agregar el nombre del cliente desde la tabla clientes
top_ticket_promedio = top_ticket_promedio.merge(clientes[['id_cliente', 'nombre_cliente']], on='id_cliente')

# Reordenar columnas para mejor visualización (opcional)
top_ticket_promedio = top_ticket_promedio[['id_cliente', 'nombre_cliente', 'ticket_promedio']]


# Mostrar tabla bonita con formato monetario y título (en Jupyter o similar)
top_ticket_promedio.style.format({'ticket_promedio': '${:,.2f}'}).set_caption("💵 Top 5 Clientes por Ticket Promedio")


,id_cliente,nombre_cliente,ticket_promedio
0,52,Gonzalo Montes,$80.66
1,47,Rubén Aguado,$76.56
2,27,Vicente Cordero,$75.41
3,37,Iván Castillo,$72.62
4,42,Carmen Mejía,$71.00
